# Physics-Informed Neural Networks

## Allan Peter Engsig-Karup (apek@dtu.dk)
## May 7th, 2024

In [2]:
import notebook as nb
print(nb.__version__)

6.3.0


In [3]:
import types
def imports():  
    
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__ 
list(imports())

['builtins', 'builtins', 'numpy', 'matplotlib.pyplot', 'notebook', 'types']

We consider in the following the use of Physics-Informed Neural Networks(PINNs) for the construction of a surrogate model for the solution to the linear advection equation in one space dimension.
$u_{t}+cu_{x}=0 \qquad  t ∈ \mathbb{R^+}, x ∈ \mathbb{R}$ 
This problem have analytical solutions in the form $u(x,t)=f(x-ct)$, and an initial condition of the form $u(x,0)=f(x)=\sin(\pi x)$ is assumed. For $c>0$, a boundary condition is imposed on the left boundary because the information is travelling from left to right in this case
$u(0,t)=f(-ct)=\sin(-\pi ct)$

In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# High precision
torch.set_default_dtype(torch.float64)

# Define device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA on GPU")
# Check for MPS availability next if CUDA is not available (for macOS with Apple Silicon)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS on Apple Silicon GPU")
    torch.set_default_dtype(torch.float32) # mps does not allow for float64
# Default to CPU if neither CUDA nor MPS is available
else:
    device = torch.device("cpu")
    print("Using CPU")

# Define the neural network model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 30),
            nn.Tanh(),
            nn.Linear(30, 30),
            nn.Tanh(),
            nn.Linear(30, 30),
            nn.Tanh(),
            nn.Linear(30, 30),
            nn.Tanh(),
            nn.Linear(30, 30),
            nn.Tanh(),
            nn.Linear(30, 1)
        )

    def forward(self, x, t):
        xt = torch.cat((x, t), dim=1)
        u = self.net(xt)
        return u

# Helper functions to calculate derivatives
def grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True, retain_graph=True)[0]

# Physics-informed loss
def loss_fn(model, x, t, c):
    x.requires_grad_(True)
    t.requires_grad_(True)

    u = model(x, t)

    u_t = grad(u, t)
    u_x = grad(u, x)

    # PDE loss
    f = u_t + c * u_x  # Ensure 'c' is defined or passed to the function

    # Boundary conditions (ensure these are correct for your problem) 
    boundary_condition = model(-1*torch.ones_like(x),t) - model(torch.ones_like(x),t)
    initial_condition = model(x, torch.zeros_like(t)) - torch.sin(2*np.pi *x)

    # Compute MSE for PDE, boundary, and initial conditions
    loss_pde = torch.mean(f**2)
    loss_boundary = torch.mean(boundary_condition**2)
    loss_initial = torch.mean(initial_condition**2)

    return loss_pde + 10.0*loss_boundary + 10.0*loss_initial
    
# Training
def train(model, epochs, optimizer, x, t,c):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        l = loss_fn(model, x, t,c)
        l.backward()
        optimizer.step()
        if epoch % 50 == 0:
            print(f'Epoch {epoch}: Loss = {l.item()}')

# Grid points
xmin, xmax, nx = -1, 1, 100
tmin, tmax, nt = 0, 2, 100
x = torch.linspace(xmin, xmax, nx, device=device).unsqueeze(1)
t = torch.linspace(tmin, tmax, nt, device=device).unsqueeze(1)
x_mesh, t_mesh = torch.meshgrid(x.flatten(), t.flatten())
x_mesh, t_mesh = x_mesh.flatten().unsqueeze(1), t_mesh.flatten().unsqueeze(1)
 
# Constants
c = 0.5

# Initialize model, optimizer
model = PINN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
train(model, 10000, optimizer, x_mesh, t_mesh,c)


Using CPU


c:\Users\maria\.conda\envs\DataAlgo\lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch 0: Loss = 5.803065353345694
Epoch 50: Loss = 3.96571928261779
Epoch 100: Loss = 0.6997458718383641
Epoch 150: Loss = 0.5076392223917112
Epoch 200: Loss = 0.2231981317539948
Epoch 250: Loss = 0.1388584598235322
Epoch 300: Loss = 0.25847352411048413
Epoch 350: Loss = 0.11616643486738304
Epoch 400: Loss = 0.17443890761400965
Epoch 450: Loss = 0.10355665022247659
Epoch 500: Loss = 0.18310164521827713
Epoch 550: Loss = 0.09098376662871573
Epoch 600: Loss = 0.5025870572881548
Epoch 650: Loss = 0.10269908905086209
Epoch 700: Loss = 0.0807760466876368
Epoch 750: Loss = 0.0574288038427526
Epoch 800: Loss = 0.430984636663023
Epoch 850: Loss = 0.09296736034289096
Epoch 900: Loss = 0.06433357705087554
Epoch 950: Loss = 0.03826164662394896


In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Assuming 'model' and 'device' are already defined and the model is trained.

# Generate predictions from the trained model
model.eval()  # Set the model to evaluation mode 
with torch.no_grad():
    u_pred = model(x_mesh, t_mesh).cpu().numpy()

# Exact solution (update this according to your problem's exact solution)
u_exact = np.sin( 2*np.pi * (x_mesh.cpu().detach().numpy() - c*t_mesh.cpu().detach().numpy()))
                 
# Compute L2 errors
l2_err = np.sum((u_exact - u_pred)**2)
l2_err_t0 = np.sum((u_exact[0, :] - u_pred[0, :])**2)
l2_err_tmax = np.sum((u_exact[-1, :] - u_pred[-1, :])**2)

# Output error information
print(f'L2 err, all timesteps: {l2_err}')
print(f'Avg-L2 err, all timesteps: {l2_err/u_pred.size}')
print(f'L2 err, t=0: {l2_err_t0}')
print(f'L2 err, t={tmax}: {l2_err_tmax}')


L2 err, all timesteps: 315.3835426443291
Avg-L2 err, all timesteps: 0.03153835426443291
L2 err, t=0: 0.00029576472218680093
L2 err, t=2: 0.18259183417764552


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming your x_mesh and t_mesh are defined correctly in the tensor form.
x = torch.linspace(xmin, xmax, nx, device=device).cpu().numpy()
t = torch.linspace(tmin, tmax, nt, device=device).cpu().numpy()

# Since these are grid centers, calculate the step size and expand the grid to cover edges.
x_step = (x[1] - x[0])
t_step = (t[1] - t[0])

# Create new arrays that represent the edges, not the centers
x_edges = np.append(x, x[-1] + x_step) - x_step/2
t_edges = np.append(t, t[-1] + t_step) - t_step/2

# Use these arrays for pcolormesh
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6, 15))

# Exact solution
ax = axes[0]
cc = ax.pcolormesh(x_edges, t_edges, u_exact.reshape((nt, nx)).T, cmap='seismic', shading='auto')
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_title('$u(x,t) (exact)$')
fig.colorbar(cc, ax=ax)

# Predicted solution
ax = axes[1]
cc = ax.pcolormesh(x_edges, t_edges, u_pred.reshape((nt, nx)).T, cmap='seismic', shading='auto')
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_title('$NN(x,t)\simeq u(x,t) (prediction)$')
fig.colorbar(cc, ax=ax)

# Errors
ax = axes[2]
cc = ax.pcolormesh(x_edges, t_edges, (u_exact.reshape((nt, nx)) - u_pred.reshape((nt, nx))).T, cmap='seismic', shading='auto')
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_title('Error')
fig.colorbar(cc, ax=ax)

NameError: name 'torch' is not defined